# Deception Detection

In [1]:
# import torch
# print(torch.__version__)
# print(torch.version.cuda)
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.7 MB/s eta 0:00:0000:01


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm.auto as tqdm
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"
from transformers import BartForConditionalGeneration
from transformers import BertTokenizer, BartModel, BartForConditionalGeneration
from torch_geometric.nn import GATConv
import torch_geometric
from torch_geometric.data import Data as GeoData
from torch.utils.data import Dataset, DataLoader
import networkx as nx
import numpy as np
import random as random
import json

# Set seeds as in the original code base . 
torch.manual_seed(1994)
np.random.seed(1994)
random.seed(1994)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

2025-04-10 14:48:14.856331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744296495.034804      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744296495.087777      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


In [3]:
# Data paths 
TRAIN_PATH = r"/kaggle/input/quanta-diplomacy/train.jsonl"
VAL_PATH = r"/kaggle/input/quanta-diplomacy/validation.jsonl"
TEST_PATH = r"/kaggle/input/quanta-diplomacy/test.jsonl"

## Custom Dataset Class

In [4]:
class ConversationDataset(Dataset):
    def __init__(self, path, max_tokens_per_msg=50, max_messages=50, use_game_scores=True):
        """
        Args:
            path (str): Path to the JSONL file
            max_tokens_per_msg (int): Maximum tokens per message
            max_messages (int): Maximum messages per conversation
            use_game_scores (bool): Whether to use game scores
        """
        super().__init__()
        self.data = []
        self.max_tokens_per_msg = max_tokens_per_msg
        self.max_messages = max_messages
        self.use_game_scores = use_game_scores
        
        # Read and process each line in the JSONL file
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                record = json.loads(line)
                messages = record.get("messages", [])
                labels = record.get("sender_labels", [])
                game_scores = record.get("game_score_delta", None) if use_game_scores else None
                
                filtered_msgs, filtered_lbls = [], []
                filtered_scores = []
                
                if game_scores is None:
                    game_scores = [0] * len(messages)
                
                for m, l, g in zip(messages, labels, game_scores):
                    if l in [True, False, "true", "false", "True", "False"]:
                        # Ensure messages are strings
                        if isinstance(m, dict) and "text" in m:
                            m = m["text"]
                        elif not isinstance(m, str):
                            m = str(m)
                        
                        filtered_msgs.append(m)
                        if isinstance(l, str):
                            filtered_lbls.append(1 if l.lower() == "true" else 0)
                        else:
                            filtered_lbls.append(1 if l else 0)
                        
                        # Convert score to float
                        try:
                            if isinstance(g, (str, bool)):
                                g = float(g) if g and g.strip() and g.lower() != "false" else 0.0
                            elif g is None:
                                g = 0.0
                            else:
                                g = float(g)
                        except (ValueError, TypeError):
                            g = 0.0
                            
                        filtered_scores.append(g)
                
                if len(filtered_msgs) == 0:
                    continue
                
                # Limit number of messages if needed
                if len(filtered_msgs) > self.max_messages:
                    filtered_msgs = filtered_msgs[:self.max_messages]
                    filtered_lbls = filtered_lbls[:self.max_messages]
                    filtered_scores = filtered_scores[:self.max_messages]
                
                # Create edges for the conversation graph
                edges = [(i, i + 1) for i in range(len(filtered_msgs) - 1)]
                
                self.data.append({
                    'messages': filtered_msgs,
                    'labels': filtered_lbls,
                    'scores': filtered_scores,
                    'edges': edges
                })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # Ensure scores are all float values
        scores = [float(s) if not isinstance(s, float) else s for s in item['scores']]
        
        return {
            'messages': item['messages'],
            'labels': torch.tensor(item['labels'], dtype=torch.long),
            'scores': torch.tensor(scores, dtype=torch.float),
            'edge_index': torch.tensor(item['edges'], dtype=torch.long).t().contiguous() if item['edges'] else torch.zeros((2, 0), dtype=torch.long)
        }

def collate_fn(batch):
    # Get batch size
    batch_size = len(batch)
    
    # Get max number of messages in this batch
    max_msgs = max(len(item['messages']) for item in batch)
    
    # Initialize lists for batch items
    all_messages = []
    all_labels = []
    all_scores = []
    attention_masks = []
    edge_indices = []
    batch_indices = []
    
    # Process each item in the batch
    for batch_idx, item in enumerate(batch):
        num_msgs = len(item['messages'])
        
        # Add messages
        all_messages.extend(item['messages'])
        
        # Add padding indicators to attention mask (1 for real message, 0 for padding)
        attention_masks.append([1] * num_msgs + [0] * (max_msgs - num_msgs))
        
        # Pad and add labels
        labels = item['labels'].tolist() + [0] * (max_msgs - num_msgs)
        all_labels.append(labels)
        
        # Pad and add scores
        scores = item['scores'].tolist() + [0.0] * (max_msgs - num_msgs)
        all_scores.append(scores)
        
        # Add edge indices with batch offset
        if item['edge_index'].size(1) > 0:  # Only if there are edges
            edges = item['edge_index'].clone()
            edges[0] += batch_idx * max_msgs  # Add batch offset
            edges[1] += batch_idx * max_msgs
            edge_indices.append(edges)
        
        # Add batch indices for each message
        batch_indices.extend([batch_idx] * num_msgs)
        batch_indices.extend([-1] * (max_msgs - num_msgs))  # Use -1 to mark padding
    
    # Combine edge indices
    edge_index = torch.cat(edge_indices, dim=1) if edge_indices else torch.zeros((2, 0), dtype=torch.long)
    
    # Create batch dictionary
    result = {
        'messages': all_messages,  # List of strings, flattened
        'labels': torch.tensor(all_labels, dtype=torch.long),  # [batch_size, max_msgs]
        'scores': torch.tensor(all_scores, dtype=torch.float),  # [batch_size, max_msgs]
        'attention_mask': torch.tensor(attention_masks, dtype=torch.long),  # [batch_size, max_msgs]
        'batch_indices': torch.tensor(batch_indices, dtype=torch.long),  # [batch_size * max_msgs]
        'edge_index': edge_index,  # [2, num_edges]
        'batch_size': batch_size,
        'max_msgs': max_msgs
    }
    
    return result

## Message Encoder

In [5]:
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [6]:
class MessageEncoder(nn.Module):
    """
    Encodes each message using a 6-layer Transformer Encoder.
    """
    def __init__(self, hidden_dim=256, n_heads=8, ff_dim=512, num_layers=6, max_len=128):
        super().__init__()
        self.max_len = max_len
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.vocab_size = len(self.tokenizer.vocab)
        self.embedding = nn.Embedding(self.vocab_size, hidden_dim)
        self.pos_encoder = PositionalEncoding(hidden_dim, dropout=0.1, max_len=max_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=n_heads, dim_feedforward=ff_dim, dropout=0.1
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.pool = nn.AdaptiveMaxPool1d(1)

    def forward(self, texts):
        # Tokenize
        toks = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=self.max_len)
        
        # Move tokenized inputs to the same device as the embedding layer
        device = self.embedding.weight.device
        input_ids = toks['input_ids'].to(device)
        attention_mask = toks['attention_mask'].to(device)
        
        x = self.embedding(input_ids)  # (B, L, D)
        
        # Transformer expects (L, B, D)
        x = x.permute(1, 0, 2)
        
        # Add positional encoding
        x = self.pos_encoder(x)
        
        x = self.transformer(x)  # (L, B, D)
        x = x.permute(1, 2, 0)    # (B, D, L)
        
        # Pool over sequence
        x = self.pool(x).squeeze(-1)  # (B, D)
        return x, attention_mask

## Conversation Graph with World Knowledge

In [7]:
class ConversationGAT(nn.Module):
    def __init__(self, in_dim, hidden_dim=128, heads=4, layers=3):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GATConv(in_dim + 1, hidden_dim, heads=heads, dropout=0.2))
        
        for _ in range(layers-2):
            self.layers.append(GATConv(hidden_dim*heads, hidden_dim, heads=heads, dropout=0.2))
            
        self.layers.append(GATConv(hidden_dim*heads, hidden_dim, heads=1, concat=False, dropout=0.2))
        self.norm = nn.LayerNorm(hidden_dim)
        
    def forward(self, node_feats, edge_index, power_deltas):
        pd = power_deltas.unsqueeze(-1)
        pd = pd.squeeze(0)
        x = torch.cat([node_feats, pd], dim=-1)
        
        for i, layer in enumerate(self.layers[:-1]):
            x = F.elu(layer(x, edge_index))
            x = F.dropout(x, p=0.2, training=self.training)
            
        x = self.layers[-1](x, edge_index)
        return self.norm(x)

## Dececption Classifier

In [8]:
class PolicyNetwork(nn.Module):
    def __init__(self, in_dim, hidden_dim=256):
        super().__init__()
        self.in_dim = in_dim
        self.attention = nn.MultiheadAttention(embed_dim=in_dim, num_heads=4, batch_first=False)
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, mask=None):
        # Check dimensions and reshape if needed
        if len(x.shape) == 2:  # If x is [batch_size, features]
            x = x.unsqueeze(0)  # Make it [1, batch_size, features]
            
        # Ensure x is [seq_len, batch_size, features]
        if x.shape[-1] != self.in_dim:
            raise ValueError(f"Expected features dimension {self.in_dim}, got {x.shape[-1]}")
            
        # Self-attention for context (x should be [seq_len, batch, dim])
        attn_out, _ = self.attention(x, x, x)
        x = x + attn_out  # Residual connection
        
        # Take the first token's representation or mean
        x = x.mean(dim=0)  # [batch, dim]
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

## Generator using BART Decoder based on word context, world knowledge, deception label and power score

In [9]:
class ResponseGenerator(nn.Module):
    def __init__(self, bart_model_name='facebook/bart-base'):
        super().__init__()
        self.tokenizer = BartTokenizer.from_pretrained(bart_model_name)
        self.decoder = BartForConditionalGeneration.from_pretrained(bart_model_name)

    def forward(self, encoder_outputs, truth_labels, power_deltas, concept_feats=None, max_length=50):
        prompts = []
        for t, pd in zip(truth_labels, power_deltas):
            label_str = 'Truth' if t == 1 else 'Lie'
            prompts.append(f"[{label_str}|Delta:{pd.item():.2f}]")
        toks = self.tokenizer(prompts, return_tensors='pt', padding=True)
        out = self.decoder.generate(
            input_ids=toks['input_ids'],
            attention_mask=toks['attention_mask'],
            encoder_outputs=(encoder_outputs.unsqueeze(0),),
            max_length=max_length
        )
        return [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in out]

## Full Deception Detection Model

In [10]:
class DeceptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = MessageEncoder()
        self.gat = ConversationGAT(in_dim=256)
        self.policy = PolicyNetwork(in_dim=128)
        # self.generator = ResponseGenerator()

    def forward(self, messages, edge_index, power_deltas, truth_labels=None, rl_mode=False):
        # 1) Encode messages with BART encoder
        msg_feats, attn_mask = self.encoder(messages)
        # 2) GAT with power deltas
        node_feats = self.gat(msg_feats, edge_index, power_deltas)
        # 3) Policy head
        logits = self.policy(node_feats)
        probs = F.softmax(logits, dim=-1)

        actions, log_probs = None, None
        if rl_mode:
            dist = torch.distributions.Categorical(probs)
            actions = dist.sample()
            log_probs = dist.log_prob(actions)

        responses = None
        # if truth_labels is not None:
        #     responses = self.generator(node_feats, truth_labels, power_deltas)

        if rl_mode:
            return probs, actions, log_probs, responses
        return probs, responses

## Training and Evaluation

In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm

def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.1):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    
    criterion = nn.CrossEntropyLoss()
    
    best_val_loss = float('inf')
    
    # Create a progress bar for epochs
    epoch_pbar = tqdm(range(num_epochs), desc="Training", position=0)
    
    for epoch in epoch_pbar:
        # Training phase
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        # Create a progress bar for train batches
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", 
                          leave=False, position=1)
        
        for batch in train_pbar:
            messages = batch['messages']
            labels = batch['labels'].to(device)
            scores = batch['scores'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Create edge indices for the conversation graph
            edge_index = batch['edge_index'].to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            probs, _ = model(messages, edge_index, scores)
            
            # Apply mask to loss computation
            loss = criterion(probs.view(-1, 2), labels.view(-1))
            loss = (loss * attention_mask.view(-1)).sum() / attention_mask.sum()
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(probs, 1)
            train_total += attention_mask.sum().item()
            train_correct += ((predicted == labels) * attention_mask).sum().item()
            
            # Update progress bar with current loss
            train_pbar.set_postfix({"loss": f"{loss.item():.4f}"})
        
        train_loss = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        # Create a progress bar for validation batches
        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", 
                        leave=False, position=1)
        
        with torch.no_grad():
            for batch in val_pbar:
                messages = batch['messages']
                labels = batch['labels'].to(device)
                scores = batch['scores'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                edge_index = batch['edge_index'].to(device)
                
                probs, _ = model(messages, edge_index, scores)
                
                loss = criterion(probs.view(-1, 2), labels.view(-1))
                loss = (loss * attention_mask.view(-1)).sum() / attention_mask.sum()
                
                val_loss += loss.item()
                _, predicted = torch.max(probs, 1)
                val_total += attention_mask.sum().item()
                val_correct += ((predicted == labels) * attention_mask).sum().item()
                
                # Update progress bar with current loss
                val_pbar.set_postfix({"loss": f"{loss.item():.4f}"})
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total

        # In training loop after validation
        scheduler.step(val_loss)
        
        # Update the epoch progress bar
        epoch_pbar.set_postfix({
            "train_loss": f"{train_loss:.4f}",
            "train_acc": f"{train_acc:.2f}%",
            "val_loss": f"{val_loss:.4f}",
            "val_acc": f"{val_acc:.2f}%"
        })
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            epoch_pbar.write(f"Epoch {epoch+1}: Saved new best model with val_loss: {val_loss:.4f}")

def create_conversation_edges(num_messages):
    # Create edges between consecutive messages
    edges = []
    for i in range(num_messages - 1):
        edges.append([i, i + 1])
    return torch.tensor(edges, dtype=torch.long).t()

# Example usage:
if __name__ == "__main__":
    # Load datasets
    train_dataset = ConversationDataset(TRAIN_PATH)
    val_dataset = ConversationDataset(VAL_PATH)
    
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)
    
    # Initialize model
    model = DeceptionModel()
    
    # Train model
    train_model(model, train_loader, val_loader)

Training:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10 [Train]:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 1/10 [Val]:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1: Saved new best model with val_loss: 0.3794


Epoch 2/10 [Train]:   0%|          | 0/184 [00:00<?, ?it/s]

...

Epoch 8/10 [Val]:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 9/10 [Train]:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 9/10 [Val]:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 10/10 [Train]:   0%|          | 0/184 [00:00<?, ?it/s]

Epoch 10/10 [Val]:   0%|          | 0/20 [00:00<?, ?it/s]